In [ ]:
pip install langchain langchain-openai python-dotenv #langchain-anthropic

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

#llm.invoke("Tell me about langchain")

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    model="gpt-4.1-mini",
    model_provider="openai",
  
)

llm.invoke("Tell me about langchain")

In [ ]:
from langchain.messages import HumanMessage, SystemMessage, AIMessage

system_msg = SystemMessage(content="You are a helpful AI assistant that explains concepts clearly.")
human_msg = HumanMessage(content="Explain what LangChain 1.0 is in simple terms.")

messages = [system_msg, human_msg]
response = llm.invoke(messages)
print(response)


In [ ]:
conversation = [
    {"role": "system", "content": "You are a helpful assistant that translates English to French."},
    {"role": "user", "content": "Translate: I love programming."},
    {"role": "assistant", "content": "J'adore la programmation."},
    {"role": "user", "content": "Translate: I love building applications."}
]


In [ ]:
stream = llm.stream([
    SystemMessage(content="You are a concise technical writer."),
    HumanMessage(content="Write a brief summary of what makes LangChain 1.0 special.")
])

for chunk in stream:
    print(chunk.content, end='', flush=True)


In [ ]:
batch_inputs = [
    "Why do parrots have colorful feathers?",
    "How do airplanes fly?",
    "What is quantum computing?"
]

response=llm.batch(batch_inputs)
response

In [ ]:
from langchain.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Add two numbers together."""

    return a + b

@tool
def subtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b

tools = [add, subtract]
tools

In [ ]:
llm = ChatOpenAI(model="gpt-4.1-mini")
llm_withtools=llm.bind_tools(tools)

result= llm_withtools.invoke("What is the sum of 15 and 27?.")
result

In [ ]:
result.tool_calls

In [ ]:
from langchain.agents import create_agent

llm = ChatOpenAI(model="gpt-4.1-mini")

agent= create_agent(
    model=llm,
    tools=tools,
    system_prompt="you are a helpful assistant who can answers questions.",
)

result=agent.invoke({
    "messages": [HumanMessage(content="who won ipl in 2025")]
})
result

In [ ]:
result['messages']

In [ ]:
agent

In [ ]:
pip install langchain-tavily tavily

In [ ]:
from tavily import TavilyClient
import os
from dotenv import load_dotenv
from langchain_tavily import TavilySearch
load_dotenv()

#tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

# @tool
# def websearch(query: str) :
#     """Perform a web search using Tavily and return the top result."""
#     results = tavily_client.search(query=query, num_results=3)
#     return results
tavily_tool= TavilySearch(    
    num_results=3
)

tools = [add, subtract, tavily_tool]

agent= create_agent(
    model=llm,
    tools=tools,
    system_prompt="you are a helpful assistant who can answers questions.",
)

result=agent.invoke({
    "messages": [HumanMessage(content="who won ipl in 2025? and also what is 33 added to 44")]
})

In [ ]:
result['messages'][1].tool_calls

In [ ]:
result

In [ ]:
result['messages'][-1].content

In [ ]:
pip install langchain_community

In [ ]:
import sqlite3
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit

# Create a sample SQLite database
def create_sample_database():
    conn = sqlite3.connect('sample_company.db')
    cursor = conn.cursor()
    
    # Create employees table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS employees (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            department TEXT,
            salary REAL,
            hire_date DATE
        )
    ''')
    
    # Create departments table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS departments (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            budget REAL
        )
    ''')
    
    # Insert sample data
    employees_data = [
        (1, 'John Smith', 'Engineering', 75000, '2022-01-15'),
        (2, 'Jane Doe', 'Marketing', 65000, '2021-03-10'),
        (3, 'Bob Johnson', 'Engineering', 80000, '2020-07-22'),
        (4, 'Alice Brown', 'HR', 60000, '2023-02-01'),
        (5, 'Charlie Wilson', 'Marketing', 70000, '2022-11-30')
    ]
    
    departments_data = [
        (1, 'Engineering', 500000),
        (2, 'Marketing', 200000),
        (3, 'HR', 150000)
    ]
    
    cursor.executemany('INSERT OR REPLACE INTO employees VALUES (?, ?, ?, ?, ?)', employees_data)
    cursor.executemany('INSERT OR REPLACE INTO departments VALUES (?, ?, ?)', departments_data)
    
    conn.commit()
    conn.close()
    print("Sample database created successfully!")

# Create the database
create_sample_database()

In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

db=SQLDatabase.from_uri("sqlite:///sample_company.db")

toolkit=SQLDatabaseToolkit(db=db,llm=llm)
tools=toolkit.get_tools()
tools

In [ ]:
system_prompt = f"""
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {db.dialect} query to run,
then look at the results of the query and return the answer.

Unless the user specifies a specific number of examples they wish to obtain, 
always limit your query to at most 10 results.

You can order the results by a relevant column to return the most interesting 
examples in the database. Never query for all the columns from a specific table, 
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while 
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you 
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
"""


dbagent= create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt,
)

In [ ]:
result=dbagent.invoke({
    "messages": [HumanMessage(content="list the top 3 highest paid employees in the company")]
})
result['messages']

In [ ]:
result['messages'][2].content

In [ ]:
result['messages'][8].tool_calls

In [ ]:
agent = create_agent(
    model=llm,    
    system_prompt="You are a helpful assistant that can answer questions .",
)

In [ ]:
result=agent.invoke({
    "messages": [HumanMessage(content="My Name is Siva")]
})
result['messages']

In [ ]:
result=agent.invoke({
    "messages": [HumanMessage(content="What is my name?")]
})
result['messages']

In [ ]:
pip install langgraph-checkpoint-sqlite

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

con= sqlite3.connect("checkpoints.sqlite",check_same_thread = False,)



checkpointer= SqliteSaver(conn=con)

agent= create_agent(
    model=llm,    
    system_prompt="You are a helpful assistant that can answer questions .",
    checkpointer= checkpointer
)

In [ ]:

config = {"configurable": {"thread_id": "memory-demo-1"}}

result=agent.invoke({
    "messages": [HumanMessage(content="What is the capital of France?")]
} , 
config=config)


result['messages']

In [ ]:
result=agent.invoke(
    {
    "messages": [HumanMessage(content="who is the president of that country?")]
} , 
config=config)


result['messages']

In [ ]:
from langchain.agents import create_agent

In [ ]:
from langchain.agents.middleware import wrap_model_call

@wrap_model_call
def logging_middleware(request, handler):
    print("Request:", request)
    print(f"model {request.model}")
    messages= request.state.get("messages", [])
    print("Messages so far:", messages)

    response = handler(request)

    print("Response:", response)
    return response



agent= create_agent(
    model=llm,    
    tools=[add, subtract],
    system_prompt="You are a helpful assistant that can answer questions .",
    middleware=[logging_middleware]
)

In [ ]:
agent.invoke({
    "messages": [HumanMessage(content="What is 45 minus 12?")]
})

In [ ]:

from langchain.agents.middleware import  ModelRequest,ModelResponse
basic_model = ChatOpenAI(model="gpt-4.1-mini")

advanced_model = ChatOpenAI(model="gpt-4.1")

@wrap_model_call
def dynamic_model_selector(request:ModelRequest, handler)-> ModelResponse:
    messages=request.state["messages"]

    last_message= messages[-1].content

    

    complex_keywords = ["complex", "multiple", "detailed", "analysis", "explain"]

    if any(keyword in last_message.lower() for keyword in complex_keywords):
        print("Using advanced model for complex query.")
        request.override(model=advanced_model)
        #request.model = advanced_model
    else:
        print("Using basic model for simple query.")
        request.override(model=basic_model)

    response = handler(request)
    return response

In [ ]:
dynamic_agent= create_agent(
    model=basic_model,
    tools=[add, subtract],
    system_prompt="You are a helpful assistant that can answer questions .",
    middleware=[dynamic_model_selector]
)


result = dynamic_agent.invoke({
    "messages": [{"role": "user", 
                  "content": "Please provide a detailed analysis of the calculation 15  + 222 - 7"}]
})
print(result["messages"][-1].content)

In [13]:
from langchain.agents.middleware import wrap_tool_call


@tool
def divide_numbers(dividend: float, divisor: float) -> str:
    """Divide two numbers.
    
    Args:
        dividend: The number to be divided
        divisor: The number to divide by
    """
    result = dividend / divisor  # This will raise ZeroDivisionError if divisor is 0
    return f"Result: {dividend} ÷ {divisor} = {result}"

In [14]:
from langchain_core.messages import ToolMessage

@wrap_tool_call
def handle_tool_errors(request, handler):
    """Handle tool execution errors with custom messages."""
    try:

        return handler(request)
    
    except ZeroDivisionError:
        return ToolMessage(
            content="Error: Cannot divide by zero. Please provide a non-zero divisor.",
            tool_call_id=request.tool_call["id"]
        )
    except Exception as e:
        return ToolMessage(
            content=f"Tool error: An unexpected error occurred. Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )


In [22]:
agent= create_agent(
    model=basic_model,    
    tools=[divide_numbers,subtract],
    system_prompt="You are a helpful assistant that can answer questions." \
    " Use appropriate tool. Dont respond directly eventhough you know the answer.",
    middleware=[   handle_tool_errors,]
)

response=agent.invoke({
    "messages": [HumanMessage(content="What is 45 divided by (22 minus 22)?")]
})

response['messages']

[HumanMessage(content='What is 45 divided by (22 minus 22)?', additional_kwargs={}, response_metadata={}, id='54396cd4-9e87-473b-ae91-1451c07b6724'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 123, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_376a7ccef1', 'id': 'chatcmpl-Cx3gTGnMAnyLZ9JPIqybeOqZalFAz', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bb06d-bcaf-71a3-a5b1-004f20cb2588-0', tool_calls=[{'name': 'subtract', 'args': {'a': 22, 'b': 22}, 'id': 'call_lZVwxMTsMqp5PZMEko4PEK9t', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 123, 'output_toke

In [23]:
from langchain.agents.middleware import dynamic_prompt

@dynamic_prompt
def adaptive_system_prompt(request: ModelRequest) -> str:
    context=request.runtime.context

    user_role = context.get("user_role", "general")
    expertise_level = context.get("expertise_level", "beginner")

    base_prompt = "You are a helpful assistant."
    
    if user_role == "student":
        if expertise_level == "beginner":
            return f"{base_prompt} Explain concepts simply, avoid jargon, and provide step-by-step explanations."
        else:
            return f"{base_prompt} Provide detailed explanations with examples and encourage critical thinking."
    elif user_role == "teacher":
        return f"{base_prompt} Provide comprehensive information that can be used for teaching, including examples and different perspectives."
    elif user_role == "expert":
        return f"{base_prompt} Provide detailed technical responses with advanced concepts and assume deep knowledge."
    
    return base_prompt

In [24]:
from typing import TypedDict

class MyContext(TypedDict):
    user_role: str
    expertise_level: str


agent= create_agent(
    model=basic_model,    
    tools=[add, subtract],
    system_prompt="You are a helpful assistant that can answer questions." \
    " Use appropriate tool. Dont respond directly eventhough you know the answer.",
    context_schema=MyContext,
    middleware=[adaptive_system_prompt],
)

agent.invoke({
    "messages": [HumanMessage(content="How do I calculate the area of a circle?")],
},
    context= {
        "user_role": "student",
        "expertise_level": "beginner"
    }
)

{'messages': [HumanMessage(content='How do I calculate the area of a circle?', additional_kwargs={}, response_metadata={}, id='111cc36b-2d31-4db1-962d-3bd651b7c1a0'),
  AIMessage(content='To calculate the area of a circle, you need to use the formula:\n\nArea = π × r²\n\nWhere:\n- π (pi) is approximately 3.14159\n- r is the radius of the circle (the distance from the center to the edge)\n\nSteps to calculate the area:\n1. Measure the radius of the circle.\n2. Square the radius (multiply the radius by itself).\n3. Multiply the squared radius by π.\n\nFor example, if the radius is 3 units:\nArea = π × 3² = π × 9 ≈ 3.14159 × 9 ≈ 28.27 square units.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 134, 'prompt_tokens': 97, 'total_tokens': 231, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}

In [45]:
from langchain.agents.middleware import PIIMiddleware


@tool
def send_email(recipient: str, subject: str, body: str) -> str:
    """Send an email to a recipient. This is a sensitive operation that requires approval."""

    
    return f"Email sent to {recipient} with subject '{subject}' and body: {body[:50]}..."

@tool
def search_database(query: str) -> str:
    """Search the company database for information."""
    return f"Database search results for '{query}': Found 5 matching records"

@tool
def delete_record(record_id: str) -> str:
    """Delete a record from the database. This is a sensitive operation."""
    print(f"Deleting record {record_id} from the database...")
    return f"Record {record_id} has been deleted from the database"

@tool
def calculate(expression: str) -> str:
    """Calculate a mathematical expression."""
    try:
        # Simple calculator for safe expressions
        result = eval(expression.replace('^', '**'))
        return f"Result: {result}"
    except Exception as e:
        return f"Error in calculation: {str(e)}"


In [ ]:
pii_agent= create_agent(
    model=basic_model,
    tools=[send_email, search_database, delete_record, calculate],
    system_prompt="You are a helpful assistant that can answer questions.",
    middleware=[
       
           
    ]
)

In [39]:
result = pii_agent.invoke({
        "messages": [{
            "role": "user", 
            "content": "My credit card number is 4356-5678-9012-3456. Can you help me with customer service?"
        }]
    })
result['messages']

[HumanMessage(content='My credit card number is 4356-5678-9012-3456. Can you help me with customer service?', additional_kwargs={}, response_metadata={}, id='80932a50-0187-4f33-b863-53845c522f68'),
 AIMessage(content='For your privacy and security, please do not share sensitive information such as your credit card number here. How can I assist you with customer service? If you have a specific issue or question, please let me know!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 164, 'total_tokens': 208, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_376a7ccef1', 'id': 'chatcmpl-Cx4GtiLaraYHWbOZ9vpPIwOMCsOgh', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs

In [88]:
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

agent= create_agent(
    model= ChatOpenAI(model="gpt-4.1"),
    tools=[send_email, search_database, delete_record, calculate],
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on = {
                "send_email": True,
                "delete_record": True,
                "search_database": False,
                "calculate": False

            }
        )
    ],
    system_prompt="You are a helpful assistant that can answer questions. use the appropriate tool.",
    checkpointer= InMemorySaver()
)

In [92]:
config = {"configurable": {"thread_id": "safe_thread766"}}

result= agent.invoke({
    "messages": [{
        "role": "user", 
        "content": "Send an email to team@company.com about the quarterly meeting requesting availability with default subject"
    }]
},    
config=config
)

result['messages']

[HumanMessage(content='Send an email to team@company.com about the quarterly meeting requesting availability with default subject', additional_kwargs={}, response_metadata={}, id='bf307c8a-f4d8-4a1f-812c-81df171b56c6'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 160, 'total_tokens': 228, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_1a2c4a5ede', 'id': 'chatcmpl-Cx5Mk0hiX3Tfcjv9Zysc03RmlgQ7k', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bb0d0-6309-7122-b89a-77ddda4a6061-0', tool_calls=[{'name': 'send_email', 'args': {'recipient': 'team@company.com', 'subject': 'Quarterly Meeting', 'body': 'Dear Team,\n

In [93]:
result

{'messages': [HumanMessage(content='Send an email to team@company.com about the quarterly meeting requesting availability with default subject', additional_kwargs={}, response_metadata={}, id='bf307c8a-f4d8-4a1f-812c-81df171b56c6'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 160, 'total_tokens': 228, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_1a2c4a5ede', 'id': 'chatcmpl-Cx5Mk0hiX3Tfcjv9Zysc03RmlgQ7k', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bb0d0-6309-7122-b89a-77ddda4a6061-0', tool_calls=[{'name': 'send_email', 'args': {'recipient': 'team@company.com', 'subject': 'Quarterly Meeting', 'body':

In [70]:
result['messages'][1].tool_calls

[{'name': 'send_email',
  'args': {'recipient': 'team@company.com',
   'subject': 'Request for Availability: Quarterly Meeting',
   'body': 'Dear Team,\n\nWe are planning our upcoming quarterly meeting and would like to request your availability. Please reply with your available dates and times in the next week so that we can schedule the meeting at a time convenient for everyone.\n\nThank you!\n\nBest regards,\n[Your Name]'},
  'id': 'call_lYOr8dtNGBs5J1NuzuvuhVXK',
  'type': 'tool_call'}]

In [71]:
result['__interrupt__']

[Interrupt(value={'action_requests': [{'name': 'send_email', 'args': {'recipient': 'team@company.com', 'subject': 'Request for Availability: Quarterly Meeting', 'body': 'Dear Team,\n\nWe are planning our upcoming quarterly meeting and would like to request your availability. Please reply with your available dates and times in the next week so that we can schedule the meeting at a time convenient for everyone.\n\nThank you!\n\nBest regards,\n[Your Name]'}, 'description': "Tool execution requires approval\n\nTool: send_email\nArgs: {'recipient': 'team@company.com', 'subject': 'Request for Availability: Quarterly Meeting', 'body': 'Dear Team,\\n\\nWe are planning our upcoming quarterly meeting and would like to request your availability. Please reply with your available dates and times in the next week so that we can schedule the meeting at a time convenient for everyone.\\n\\nThank you!\\n\\nBest regards,\\n[Your Name]'}"}], 'review_configs': [{'action_name': 'send_email', 'allowed_decis

In [ ]:
from langgraph.types import Command
result= agent.invoke(
    Command(resume= { "decisions": [{"type": "edit"}]}),
    config=config
)

result

{'messages': [HumanMessage(content='Please delete record 12345 from the database.', additional_kwargs={}, response_metadata={}, id='6e433f95-62ef-427a-9fcb-ff10ecd66339'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 153, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_1a2c4a5ede', 'id': 'chatcmpl-Cx4wZFRXXWaQowPNsRHAfrYuF2ZFR', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bb0b7-9f87-7111-98b3-fc569b169b08-0', tool_calls=[{'name': 'delete_record', 'args': {'record_id': '12345'}, 'id': 'call_pHSBLEbOT0vZZYpUMEXTWEUT', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input

In [80]:
from langchain.agents.middleware import SummarizationMiddleware

agent= create_agent(
    model= ChatOpenAI(model="gpt-4.1"),
    tools=[send_email, search_database, delete_record, calculate],
    middleware=[
        SummarizationMiddleware(
            model= ChatOpenAI(model="gpt-4.1-mini"),
            trigger=("messages", 10),
            keep= ("messages", 4)
            
        )
    ],
    system_prompt="You are a helpful assistant that can answer questions. use the appropriate tool.",
    checkpointer= InMemorySaver()
)

In [81]:
config = {"configurable": {"thread_id": "2"}}

result= agent.invoke({
    "messages": [HumanMessage(content="MY name is Siva")]
} , config=config)
result['messages']

[HumanMessage(content='MY name is Siva', additional_kwargs={}, response_metadata={}, id='6baf2f63-cb7b-48ff-834e-cc736e84ef16'),
 AIMessage(content='Hello Siva! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 148, 'total_tokens': 160, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_1a2c4a5ede', 'id': 'chatcmpl-Cx5HsCjaydllWr0sGxTP6fZop4Yu6', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bb0cb-c762-7493-a257-c87b5357dc30-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 148, 'output_tokens': 12, 'total_tokens': 160, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_det

In [82]:
result= agent.invoke({
    "messages": [HumanMessage(content="Tell me about axa in 2 sentences")]
} , config=config)
result['messages']

[HumanMessage(content='MY name is Siva', additional_kwargs={}, response_metadata={}, id='6baf2f63-cb7b-48ff-834e-cc736e84ef16'),
 AIMessage(content='Hello Siva! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 148, 'total_tokens': 160, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_1a2c4a5ede', 'id': 'chatcmpl-Cx5HsCjaydllWr0sGxTP6fZop4Yu6', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bb0cb-c762-7493-a257-c87b5357dc30-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 148, 'output_tokens': 12, 'total_tokens': 160, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_det

In [83]:
result= agent.invoke({
    "messages": [HumanMessage(content="Tell me about india  in 2 sentences")]
} , config=config)
result['messages']

[HumanMessage(content='MY name is Siva', additional_kwargs={}, response_metadata={}, id='6baf2f63-cb7b-48ff-834e-cc736e84ef16'),
 AIMessage(content='Hello Siva! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 148, 'total_tokens': 160, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_1a2c4a5ede', 'id': 'chatcmpl-Cx5HsCjaydllWr0sGxTP6fZop4Yu6', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bb0cb-c762-7493-a257-c87b5357dc30-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 148, 'output_tokens': 12, 'total_tokens': 160, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_det

In [84]:
result= agent.invoke({
    "messages": [HumanMessage(content="Tell me about bitcoin  in 2 sentences")]
} , config=config)
result['messages']

[HumanMessage(content='MY name is Siva', additional_kwargs={}, response_metadata={}, id='6baf2f63-cb7b-48ff-834e-cc736e84ef16'),
 AIMessage(content='Hello Siva! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 148, 'total_tokens': 160, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_1a2c4a5ede', 'id': 'chatcmpl-Cx5HsCjaydllWr0sGxTP6fZop4Yu6', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bb0cb-c762-7493-a257-c87b5357dc30-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 148, 'output_tokens': 12, 'total_tokens': 160, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_det

In [85]:
result= agent.invoke({
    "messages": [HumanMessage(content="Tell me about Sivaprasad  in 2 sentences")]
} , config=config)
result['messages']

[HumanMessage(content='MY name is Siva', additional_kwargs={}, response_metadata={}, id='6baf2f63-cb7b-48ff-834e-cc736e84ef16'),
 AIMessage(content='Hello Siva! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 148, 'total_tokens': 160, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_1a2c4a5ede', 'id': 'chatcmpl-Cx5HsCjaydllWr0sGxTP6fZop4Yu6', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bb0cb-c762-7493-a257-c87b5357dc30-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 148, 'output_tokens': 12, 'total_tokens': 160, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_det

In [86]:
result= agent.invoke({
    "messages": [HumanMessage(content="Tell me about Modi  in 2 sentences")]
} , config=config)
result['messages']

[HumanMessage(content="Here is a summary of the conversation to date:\n\nSiva introduced himself. AXA is a multinational insurance company headquartered in France, offering a wide range of insurance, investment management, and financial services globally, recognized as a leading insurance brand in over 50 countries. India is the world's most populous democracy, seventh-largest by land area, known for cultural diversity and a rapidly growing economy with advancements in technology and space exploration.", additional_kwargs={}, response_metadata={}, id='57048d48-c826-40ea-a36d-30e6682daadd'),
 AIMessage(content='Bitcoin is a decentralized digital currency created in 2009 by an unknown person or group using the pseudonym Satoshi Nakamoto, allowing peer-to-peer transactions on a secure blockchain network without the need for a central authority. Its value is highly volatile and it is widely recognized as both a speculative investment asset and a potential alternative to traditional currenc

In [94]:
import time

@tool
def get_weather(city: str) -> str:
    """Get the current weather for a specified city."""
    # Simulate API call delay
    time.sleep(5)
    
    # Mock weather responses for different cities
    weather_data = {
        "new york": "Partly cloudy, 22°C with light winds",
        "london": "Rainy, 15°C with moderate winds", 
        "tokyo": "Sunny, 28°C with clear skies",
        "paris": "Overcast, 18°C with occasional showers",
        "sydney": "Sunny, 25°C with gentle breeze"
    }
    
    city_lower = city.lower()
    if city_lower in weather_data:
        return f"Weather in {city}: {weather_data[city_lower]}"
    else:
        return f"Weather in {city}: Sunny, 24°C (default forecast)"

@tool
def calculate_travel_time(origin: str, destination: str) -> str:
    """Calculate estimated travel time between two locations."""
    # Simulate calculation delay
    time.sleep(3)
    
    # Mock travel time calculation
    import random
    hours = random.randint(1, 12)
    minutes = random.randint(0, 59)
    
    return f"Estimated travel time from {origin} to {destination}: {hours}h {minutes}m by car"

In [95]:
llm = ChatOpenAI(
    model="gpt-4.1-mini",  # GPT-4.1-mini (using the correct model name)
    streaming=True,
)

tools = [get_weather, calculate_travel_time]


agent = create_agent(llm, tools)

In [97]:
query = "What's the weather like in Tokyo? Also, how long would it take to travel from Tokyo to Osaka?"

for i, chunk in enumerate(agent.stream(
    {"messages": [HumanMessage(content=query)]},
    stream_mode="updates"
), 1):
    

    print(f"📍 Step {i}:")
    for node_name, data in chunk.items():
        print(f"--- {node_name} ---")
        print(data['messages'][-1].content)

📍 Step 1:
--- model ---

📍 Step 2:
--- tools ---
Estimated travel time from Tokyo to Osaka: 3h 44m by car
📍 Step 3:
--- tools ---
Weather in Tokyo: Sunny, 28°C with clear skies
📍 Step 4:
--- model ---
The weather in Tokyo is currently sunny with clear skies and a temperature of 28°C. The estimated travel time from Tokyo to Osaka by car is approximately 3 hours and 44 minutes. Is there anything else you would like to know?


In [102]:
query = "Tell me about Elon Musk in 10 bullet points"

for chunk in agent.stream(
    {"messages": [HumanMessage(content=query)]},
    stream_mode="messages"
):
    print(chunk[0].content, end='', flush=True)
    

1. Elon Musk is a billionaire entrepreneur and business magnate known for founding, co-founding, or leading several high-profile technology companies.
2. He was born on June 28, 1971, in Pretoria, South Africa.
3. Musk is the CEO and lead designer of SpaceX, a company focused on space exploration and reducing space transportation costs.
4. He is also the CEO and product architect of Tesla, Inc., which specializes in electric vehicles and renewable energy products.
5. Elon Musk co-founded PayPal, an influential online payment system, which was later sold to eBay.
6. He has a vision of colonizing Mars to ensure humanity's survival as a multi-planetary species.
7. Musk founded Neuralink, a neurotechnology company developing brain-machine interfaces to merge humans with AI.
8. He initiated The Boring Company, which aims to reduce traffic in cities through tunnel construction for transportation.
9. Musk is known for his controversial and often candid presence on social media platforms like 

In [103]:
from langgraph.config import get_stream_writer


@tool
def analyze_market_trends(stock_symbol: str) -> str:
    """Analyze market trends for a given stock symbol with detailed progress updates."""
    writer = get_stream_writer()
    
    # Simulate a multi-step analysis process
    writer(f"📊 Starting analysis for {stock_symbol.upper()}...")
    time.sleep(5)
    
    writer(f"📈 Fetching historical price data...")
    time.sleep(5)
    
    writer(f"🔍 Analyzing volume patterns...")
    time.sleep(5)
    
    writer(f"🧮 Calculating technical indicators...")
    time.sleep(5)
    
    writer(f"📊 Generating trend analysis report...")
    time.sleep(5)
    
    writer(f"✅ Analysis complete for {stock_symbol.upper()}!")
    
    # Return the final analysis
    trends = ["Bullish momentum", "Strong volume support", "Positive outlook"]
    return f"Market analysis for {stock_symbol.upper()}: {', '.join(trends)}"

In [104]:
agent= create_agent(
    model= ChatOpenAI(model="gpt-4.1-mini"),    
    tools=[analyze_market_trends],
)

query = "Can you analyze the market trends for AAPL stock?"


for chunk in agent.stream(
    {"messages": [HumanMessage(content=query)]},
    stream_mode="custom"
):
    print(f"🔔 {chunk}")
    time.sleep(0.1) 

🔔 📊 Starting analysis for AAPL...
🔔 📈 Fetching historical price data...
🔔 🔍 Analyzing volume patterns...
🔔 🧮 Calculating technical indicators...
🔔 📊 Generating trend analysis report...
🔔 ✅ Analysis complete for AAPL!
